In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<p style = "font-size : 50px; color : #020b1a ; font-family : 'Chalkduster',fantasy; text-align : center; background-color : #b0f75e; border-radius: 5px 5px;"><strong>Diabetes Prediction</strong></p>


> Diabetes is a disease that occurs when your blood glucose, also called blood sugar, is too high. Blood glucose is your main source of energy and comes from the food you eat. Insulin, a hormone made by the pancreas, helps glucose from food get into your cells to be used for energy. Sometimes your body doesn’t make enough—or any—insulin or doesn’t use insulin well. Glucose then stays in your blood and doesn’t reach your cells.
Over time, having too much glucose in your blood can cause health problems. Although diabetes has no cure, you can take steps to manage your diabetes and stay healthy.

<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Importing required libraries :-</strong></p>

In [ ]:
#Visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap 
#Libraries for ML
from sklearn.preprocessing import StandardScaler #Standardizasyon için
from sklearn.model_selection import train_test_split, GridSearchCV #GridSearchCV: KNN ile ilgili en iyi parametreleri belirlemek
from sklearn.metrics import accuracy_score ,confusion_matrix #Sonuç değerlendirme
from sklearn.neighbors import KNeighborsClassifier,LocalOutlierFactor #Trainin algoritması ve NCA ve Outlier değerler için
from sklearn.decomposition import PCA #PCA için

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import itertools
plt.style.use('fivethirtyeight')
import seaborn as sns
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')


<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Reading Data :-</strong></p>

In [ ]:
data = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
data.head()

<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Variables: :-</strong></p>
<ul>
    <li style = "color : #03506f; font-size : 18px; font-family: system-ui;"><strong>Pregnancies: Number of times pregnant</strong></li>
    <li style = "color : #03506f; font-size : 18px; font-family: system-ui;"><strong>Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test</strong></li>
    <li style = "color : #03506f; font-size : 18px; font-family: system-ui;"><strong>BloodPressure: Diastolic blood pressure (mm Hg)</strong></li>
    <li style = "color : #03506f; font-size : 18px;font-family: system-ui;"><strong>SkinThickness: Triceps skin fold thickness (mm)</strong></li>
    <li style = "color : #03506f; font-size : 18px; font-family: system-ui;"><strong>Insulin: 2-Hour serum insulin (mu U/ml)</strong></li>
    <li style = "color : #03506f; font-size : 18px; font-family: system-ui;"><strong>BMI: Body mass index (weight in kg/(height in m)^2)</strong></li>
    <li style = "color : #03506f; font-size : 18px;font-family: system-ui;"><strong>DiabetesPedigreeFunction: Diabetes pedigree functionr</strong></li>
    <li style = "color : #03506f; font-size : 18px; font-family: system-ui;"><strong>Age: Age (years)</strong></li>
    <li style = "color : #03506f; font-size : 18px; font-family: system-ui;"><strong>Cabin : Cabin Number</strong></li>
    <li style = "color : #03506f; font-size : 18px; font-family: system-ui;;"><strong>Outcome: Class variable (0 or 1)</strong></li>
</ul>
​

In [ ]:
print('Shape of dataset is' + str(data.shape))

In [ ]:
data.info()

In [ ]:
#Checking for null values
data.isnull().sum()

Since some features are supposed to have zero value ,we replace it with

In [ ]:
data[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = data[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.nan)
data.isnull().sum()


In [ ]:
from scipy.stats import skew
for col in data.drop('Outcome', axis = 1).columns:
    print("Skewness for the column {} is {}".format(col, data[col].skew()))

> Columns like Pregnancies, Glucose, BloodPressure, SkinThickness and BMI are not that much skewed. We can fill null values with the mean for these columns, but for columns like Insulin and DiabetesPedigreeFunction, we will have to replace them will median due to the effect of skewness

In [ ]:
data['Insulin'] = data['Insulin'].fillna(data['Insulin'].median()) # Filling null values with the median.

for col in ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']:
    data[col] = data[col].fillna(data[col].mean())

In [ ]:
data.isnull().sum()

<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Exploratory Data Analysis :-</strong></p>

> CORRELATION MATRIX:Variables within a dataset can be related for lots of reasons. It can be useful in data analysis and modeling to better understand the relationships between variables. The statistical relationship between two variables is referred to as their correlation.

In [ ]:
corr_data = data.corr() 
sns.clustermap(corr_data,annot= True,cmap = "YlGnBu",fmt = '.2f')
plt.title('Correlation Between Features')
plt.show();

In [ ]:

fig=data.hist(figsize = (20,20), color='#09FBD3',alpha=0.7, rwidth=0.85)


In [ ]:
sns.set(style="whitegrid")
labels = ['Non-Diabetic', 'Diabetic']
sizes = data['Outcome'].value_counts(sort = True)

colors = ["#09FBD3","#FDF200"]
explode = (0.05,0) 
 
plt.figure(figsize=(7,7))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90,)

plt.title('Non-Diabetic vs Diabetic')
plt.show()

In [ ]:
plt.style.use('ggplot') # TO GET AN IDEA OF OUTLIERS 

f, ax = plt.subplots(figsize=(11, 15))

ax.set_facecolor('#09FBD3')
ax.set(xlim=(-.05, 200))
plt.ylabel('Variables')
plt.title("Overview")
ax = sns.boxplot(data = data, 
  orient = 'h', 
  palette = 'Set2',)

Glucose vs Age

In [ ]:
def distplot(col_name):
    
    plt.figure()
    sns.set(style="whitegrid")
    ax = sns.distplot(data[col_name][data.Outcome == 1], color ="gold", rug = True)
    sns.distplot(data[col_name][data.Outcome == 0], color ="#09FBD3",rug = True)
    plt.legend(['Diabetic', 'Healthy'])

In [ ]:

sns.set(style='whitegrid')  
g = sns.scatterplot(x='Glucose', y='Age', data=data, hue='Outcome', palette='pastel', legend=True)
plt.legend(title='Result', loc='upper left', labels=['Healthy' , 'Diabetic'])
plt.show(g)



> Young people are generally healthier ans have less glucose concentration.

In [ ]:
pallete = {0: "#09FBD3", 1: "gold"}
sns.boxplot(x = 'Outcome', y = 'Glucose', boxprops=dict(alpha=.5), data = data, palette = pallete)
plt.title('Glucose vs Outcome')
plt.show()

In [ ]:
distplot('Glucose')

>Both plots show that diabetic people tend to have a much higher glucose level.

Pregnencies

In [ ]:
distplot('Pregnancies')

> It can be said that number of pregnancies is high for the diabetic people

Insulin

In [ ]:
sns.jointplot(x='Insulin',y='Glucose', data=data, kind = 'reg', color = '#09FBD3')
plt.show()

Insulin increases with incerase of glucose level

In [ ]:
distplot('Insulin')

> Diabetic People tend to have slightly higher Insulin level.

BLood Pressure

In [ ]:
distplot('BloodPressure')

> Diabetes increases risk of high blood pressure

BMI

In [ ]:
pallete = {0: "#09FBD3", 1: "gold"}
sns.boxplot(x = 'Outcome', y = 'BMI', boxprops=dict(alpha=.5), data = data, palette = pallete)

plt.title('BMI vs Outcome')
plt.show()

> It can be seen that the Median BMI of the Diabetic People is greater than the Median BMI of the Non-Diabetic people.

<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Modelling :-</strong></p>

> Why do we need scaling?
Feature scaling is essential for machine learning algorithms that calculate distances between data. If not scale, the feature with a higher value range starts dominating when calculating distances,

In [ ]:
df = data
zero  = df[df['Outcome']==0]   #zero values in outcome column
one = df[df['Outcome']==1]  # one values in outcome column
from sklearn.utils import resample
#minority class that 1, we need to upsample/increase that class so that there is no bias
#n_samples = 500 means we want 500 sample of class 1, since there are 500 samples of class 0
df_minority_upsampled = resample(one, replace = True, n_samples = 500) 
#concatenate
df = pd.concat([zero, df_minority_upsampled])

from sklearn.utils import shuffle
df = shuffle(df) # shuffling so that there is particular sequence

In [ ]:
from sklearn.model_selection import train_test_split
#Splitting train and test data
X = df.drop('Outcome', axis = 1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42, stratify = y)


#Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
features= X.columns
X[features] = sc.fit_transform(X[features])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC



from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


In [ ]:
lr = LogisticRegression(random_state=42)

knn = KNeighborsClassifier()

dt = DecisionTreeClassifier()

rf = RandomForestClassifier()

XGB = XGBClassifier()
svc = SVC()










In [ ]:
classifiers = [('Logistic Regression', lr), ('K Nearest Neighbours', knn),
               ('Decision Tree', dt), ('Random Forest', rf),('XGBoost',XGB),('SVM',svc)]

for classifier_name, classifier in classifiers:
 
    # Fit clf to the training set
    classifier.fit(X_train, y_train)    
   
    # Predict y_pred
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    

   
    # Evaluate clf's accuracy on the test set
    print('{:s} : {:.2f}'.format(classifier_name, accuracy))

Decision Tree based models prove to be more effecient in this classification problem.